In [ ]:
# !pip install pyarrow
# !pip install fastparquet
# !pip install gensim

### Overview

（１）以前購入したもの（２）色違い（３）ペア購入商品（４）トレンド１（５）トレンド２（６）トレンド３（７）トレンド４（８）数年連続人気（９）協調フィルタリング１（１０）協調フィルタリング２の結果をそれぞれ計算し、アンサンブル。基本は予測（１）（２）（３）（９）（１０）を重視し、埋まらない部分をトレンドで埋める

In [ ]:
# val_week : 104 ... calc CV
#            105 ... make submission
# use_item2vec : True ... use item2vec ! very slow
#                False ... not use item 3 vec
# is_load : True ... load result datas
# is_save : True ... save result datas

### Libraries and Functions

In [ ]:
import numpy as np, pandas as pd, datetime as dt
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
from collections import defaultdict

def iter_to_str(iterable):
    return " ".join(map(lambda x: str(0) + str(x), iterable))

def apk(actual, predicted, k=12):
    if len(predicted) > k:
        predicted = predicted[:k]
    score, nhits = 0.0, 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            nhits += 1.0
            score += nhits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=12, return_apks=False):
    assert len(actual) == len(predicted)
    apks = [apk(ac, pr, k) for ac, pr in zip(actual, predicted) if 0 < len(ac)]
    if return_apks:
        return apks
    return np.mean(apks)

def blend(dt, w=[], k=12):
    if len(w) == 0:
        w = [1] * (len(dt))
    preds = []
    for i in range(len(w)):
        preds.append(dt[i].split())
    res = {}
    for i in range(len(preds)):
        if w[i] < 0:
            continue
        for n, v in enumerate(preds[i]):
            if v in res:
                res[v] += (w[i] / (n + 1))
            else:
                res[v] = (w[i] / (n + 1))    
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    return ' '.join(res[:k])

def prune(pred, ok_set, k=12):
    pred = pred.split()
    post = []
    for item in pred:
        if int(item) in ok_set and not item in post:
            post.append(item)
    return " ".join(post[:k])

def validation(actual, predicted, grouping, score=0, index=-1, ignore=False, figsize=(12, 6)):
    # actual, predicted : list of lists
    # group : pandas Series
    # score : pandas DataFrame
    
    vc = pd.Series(predicted).apply(len).value_counts()
    print("Fill Rate = ", round(1 - sum(vc[k] * (12 - k) / 12 for k in (set(range(12)) & set(vc.index))) / len(actual), 3) * 100)
    
    
    if ignore: return
    ap12 = mapk(actual, predicted, return_apks=True)
    map12 = round(np.mean(ap12), 6)
    if isinstance(score, int): score = pd.DataFrame({g:[] for g in sorted(grouping.unique().tolist())})
    if index == -1 : index = score.shape[0]
    score.loc[index, "All"] = map12
    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1); sns.histplot(data=ap12, log_scale=(0, 10), bins=20); plt.title(f"MAP@12 : {map12}")
    for g in grouping.unique():
        map12 = round(mapk(actual[grouping == g], predicted[grouping == g]), 6)
        score.loc[index, g] = map12
    plt.subplot(1, 2, 2); score[[g for g in grouping.unique()[::-1]] + ['All']].loc[index].plot.barh(); plt.title(f"MAP@12 of Groups")
    vc = pd.Series(predicted).apply(len).value_counts()
    score.loc[index, "Fill"] = round(1 - sum(vc[k] * (12 - k) / 12 for k in (set(range(12)) & set(vc.index))) / len(actual), 3) * 100
    display(score)
    return score

In [ ]:
from tqdm import tqdm

def sort_item2vec(colname, max_n = 2, none = "") :
    l = []
    for i in tqdm(range(len(sub))) :
        wid = sub.iloc[i]['customer_id']
        oc = sub.iloc[i][colname].split()
        if len(oc) == 0 :
            l.append(none)
        elif wid not in cust_buy :
            l.append(" ".join(oc))            
        elif len(oc) <= 1 :
            l.append(" ".join(oc))
        else :
            ll = []
            for e in oc : 
                buy = cust_buy[wid].split(" ")
                p = model.wv.similarity(buy, e).mean()
                ll.append([p, e])
            ll.sort(reverse=True)
            l.append(" ".join(list(map(lambda x: x[1], ll[:max_n]))))
    #     if i == 10: break

    return l


# 負のものを削除
def remove_item2vec(colname, max_n = 2, none = "") :
    l = []
    for i in tqdm(range(len(sub))) :
        wid = sub.iloc[i]['customer_id']
        oc = sub.iloc[i][colname].split()
        if len(oc) == 0 :
            l.append(none)
        elif wid not in cust_buy :
            l.append(" ".join(oc))            
        elif len(oc) <= 1 :
            l.append(" ".join(oc))
        else :
            ll = []
            rr = []
            for e in oc : 
                buy = cust_buy[wid].split(" ")
                p = model.wv.similarity(buy, e).mean()
                if p < 0 : 
                    rr.append(e)
                else :
                    ll.append(e)
            l.append(" ".join(ll+rr))
    #     if i == 10: break

    return l


def cut_items(name, n = 12) :
    l = []
    for i in tqdm(range(len(sub))) :
        s = sub[name][i].split(" ")
        l.append(" ".join(s[:n]))
    return l

In [ ]:
val_week = 104
use_item2vec = False
is_load = False
is_save = False

### wod2vec

In [ ]:
import json
from  gensim import models

if val_week == 104 :
    print("item2vec = 104")
    model = models.Word2Vec.load('../input/handmitem2vec/word2vec_v2_week104.model')
    with open('../input/handmitem2vec/cust_buy104.json', "r") as f :
        cust_buy = json.load(f)
else :
    print("item2vec = 105")
    model = models.Word2Vec.load('../input/handmitem2vec/word2vec_v2.model')
    with open('../input/handmitem2vec/cust_buy.json', "r") as f :
        cust_buy = json.load(f)


### Data

In [ ]:
# cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
# df = pd.read_parquet('../input/hm-parquets-of-datasets/transactions_train.parquet')
# cdf = cdf[['customer_id', 'attribute']]
# df = df.merge(cdf, on='customer_id', how='left')
# df = df[df['attribute'] == 'Woman']
# df

In [ ]:
df = pd.read_parquet('../input/hm-parquets-of-datasets/transactions_train.parquet')
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
cid = pd.DataFrame(sub.customer_id.apply(lambda s: int(s[-16:], 16)))

### Kangol x H&Mを除外（val_weekが105の場合）

In [ ]:
# if val_week == 105 :
#     print("remove Kangol x H&M")
#     articles_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv",dtype=str, encoding='utf8')
#     xHM = []
#     for i, e in enumerate(articles_df['detail_desc'].fillna("")) :
#         if 'Kangol x H&M' in e:
#             #print(articles_df.iloc[i]['article_id'], e)
#             xHM.append(int(articles_df.iloc[i]['article_id']))

#     #xHM
#     df = df[~df.article_id.isin(xHM)]

### Definition of Groups

In [ ]:
group = df.groupby('customer_id').sales_channel_id.mean().round().reset_index()\
    .merge(cid, on='customer_id', how='right').rename(columns={'sales_channel_id':'group'})
grouping = group.group.fillna(1.0)

### One-Week Hold Out

In [ ]:
# id of week to be used in a validation; set 105 if you would like to create a submission
val = df.loc[df.week == val_week].groupby('customer_id').article_id.apply(iter_to_str).reset_index()\
    .merge(cid, on='customer_id', how='right')
actual = val.article_id.apply(lambda s: [] if pd.isna(s) else s.split())
last_date = df.loc[df.week < val_week].t_dat.max()

In [ ]:
last_date

## load Result

In [ ]:
if is_load :
    print("LOAD...")
    if val_week == 105: 
        sub = pd.read_parquet('submission_all.parquet')
    else :
        sub = pd.read_parquet('submission_all_104.parquet')
    predicted = sub['last_purchase'].apply(lambda s: [] if pd.isna(s) else s.split())
    score = validation(actual, predicted, grouping, index='Last Purchase', ignore=(val_week == 105))
        
sub

# ======================

### 最後に購入したアイテム(14日前から）

In [ ]:
init_date = last_date - dt.timedelta(days=9999)
train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()
train = train.merge(train.groupby('customer_id').t_dat.max().reset_index().rename(columns={'t_dat':'l_dat'}),
                   on = 'customer_id', how='left')
train['d_dat'] = (train.l_dat - train.t_dat).dt.days
train = train.loc[train.d_dat < 14].sort_values(['t_dat'], ascending=False).drop_duplicates(['customer_id', 'article_id'])
#train = train.loc[train.d_dat < 9999].sort_values(['t_dat'], ascending=False).drop_duplicates(['customer_id', 'article_id'])
sub['last_purchase'] = train.groupby('customer_id')\
    .article_id.apply(iter_to_str).reset_index()\
    .merge(cid, on='customer_id', how='right').article_id.fillna('')

In [ ]:
# sub['last_purchase'] = cut_items('last_purchase', n = 6)

In [ ]:
# if use_item2vec:
#     sub['last_purchase'] = swap_item2vec('last_purchase')

In [ ]:
predicted = sub['last_purchase'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, index='Last Purchase', ignore=(val_week == 105))
#Last Purchase	0.012392	0.023801	0.020029	26.9
#Last Purchase	0.012807	0.024439	0.020594	28.7
#Last Purchase	0.011057	0.020228	0.017196	14.2
#Last Purchase	0.010913	0.020194	0.017126	14.2 item2vec
#Last Purchase	0.012333	0.023044	0.019503	21.4

### 購入したアイテムの他の色（６日前まで）

In [ ]:
init_date = last_date - dt.timedelta(days=6)
#init_date = last_date - dt.timedelta(days=14)
train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()\
    .groupby(['article_id']).t_dat.count().reset_index()
adf = pd.read_parquet('../input/hm-parquets-of-datasets/articles.parquet')
adf = adf.merge(train, on='article_id', how='left').rename(columns={'t_dat':'ct'})\
    .sort_values('ct', ascending=False).query('ct > 0')

map_to_col = defaultdict(list)
for aid in adf.article_id.tolist():
    map_to_col[aid] = list(filter(lambda x: x != aid, adf[adf.product_code == aid // 1000].article_id.tolist()))[:1]

def map_to_variation(s):
    f = lambda item: iter_to_str(map_to_col[int(item)])
    return ' '.join(map(f, s.split()))
sub['other_colors'] = sub['last_purchase'].fillna('').apply(map_to_variation)

In [ ]:
# # 11日間の売れた商品リストに入っていないアイテムは削除する
init_date = last_date - dt.timedelta(days=11)
sold_set = set(df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].article_id.tolist())

sub['other_colors'] = sub['other_colors'].apply(prune, ok_set=sold_set)

# l = []
# for i in tqdm(range(len(sub))) :
#     itms = sub['last_purchase'][i].split(" ")
#     t = set()
#     for e in itms:
# #         print(e)
#         if e == "" : continue
#         if int(e) not in sold_set : continue
#         t.add(e)
# #     break
#     l.append(" ".join(list(t)))

# sub['last_purchase'] = l

In [ ]:
if use_item2vec:
    sub['other_colors'] = sort_item2vec('other_colors', max_n=6)

In [ ]:
# sub['other_colors'] = cut_items('other_colors', n=2)

In [ ]:
predicted = sub['other_colors'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Other Colors', ignore=(val_week == 105))
#0.008075(6) -> 0.007619(3) -> 0.007058(2) -> 0.005448(1)

### ★ペアで購入した商品

In [ ]:
init_date = last_date - dt.timedelta(days=28)
train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()
u_art = train['article_id'].unique()
len(u_art)

In [ ]:
train = train.groupby(['t_dat', 'customer_id'], as_index=False)['article_id'].agg(list)
train

In [ ]:
m = defaultdict(int)
for e in train['article_id'] :
    l = []
    for v in e :
        l.append(v)
    l.sort()
    for i in range(len(l)) :
        for j in range(i+1, len(l)) :
            m[(l[i], l[j])]+=1
            
mm = defaultdict(list)
for e in m :
    mm[e[0]].append((m[e], e[1]))

for e in mm :
    mm[e].sort(reverse=True)
    
len(mm)

In [ ]:
th = 5
l = []
for i in tqdm(range(len(sub))) :
#     ci = sub['customer_id'][i]
#     buy = []
#     if ci in cust_buy :
#         buy = cust_buy[ci].split(" ")
    lp = sub['last_purchase'][i].split(" ")
    t = defaultdict(int)
    if len(lp) != 0 :
        for e in lp :
            if e == '' : break
            j = int(e)
            for v in mm[j][:24] :
                if v[0] < th : break # スコアが１以下はスキップ
                aid = "0" + str(v[1])
#                 if aid in buy : continue
                t[aid] += v[0]
    t = list(t.items())
    t = sorted(t, key=lambda x: x[0] in t)
    t = [x[0] for x in t]
    l.append(" ".join(t[:12]))
#     if i == 2: break
l[:10]

In [ ]:
sub['buy_together'] = l
sub.head(5)

In [ ]:
predicted = sub['buy_together'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Buy Together', ignore=(val_week == 105))
#0.014198 0.014810 0.015348 0.015676

### 人気商品 ：年齢層、オンライン／オフライン（４日前まで）グループは1,2,na<-1

In [ ]:
# cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
# cdf.fashion_news_frequency.unique()
# cdf.head()

In [ ]:
cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
cdf = cdf[['customer_id','age','attribute']]
#listBin = [-1, 19, 29, 39, 49, 59, 69, 119]
listBin = [-1, 19, 34, 49, 119]
#cdf['cust_feature'] = pd.cut(cdf['age'], listBin).astype(str).str.cat(cdf['attribute'].astype(str))
#cdf['cust_feature'] = pd.qcut(cdf['age'], 10).astype(str).str.cat(cdf['attribute'].astype(str))
#cdf['cust_feature'] = pd.cut(cdf['age'], listBin).astype(str)
cdf['cust_feature'] = pd.qcut(cdf['age'], 10).astype(str)
cdf = cdf.drop('age', axis=1)
cdf = cdf.drop('attribute', axis=1)
cdf['cust_feature'].unique()

In [ ]:
# # 4日間の売れた商品リスト
# init_date = last_date - dt.timedelta(days=4)
# sold_set = set(df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].article_id.tolist())
# len(sold_set)

In [ ]:
init_date = last_date - dt.timedelta(days=5 - 1)
group_df = pd.concat([cid, group.group.fillna(1)], axis=1) # grouping can be changed
group_df = group_df.merge(cdf, on='customer_id', how='right')
group_df.columns = ['customer_id', 'group', 'cust_feature']
group_df['cust_feature'] = group_df['group'].astype(str).str.cat(group_df['cust_feature'])
train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()\
    .merge(group_df, on='customer_id', how='left')\
    .groupby(['cust_feature', 'article_id']).t_dat.count().reset_index()

items = defaultdict(str)
for g in train.cust_feature.unique():
#    items[g] = iter_to_str(train.loc[train.cust_feature == g].sort_values('t_dat', ascending=False).article_id.tolist()[:30])
    items[g] = iter_to_str(train.loc[train.cust_feature == g].sort_values('t_dat', ascending=False).article_id.tolist()[:20])

sub['trend_items'] = group_df.cust_feature.map(items)

In [ ]:
if use_item2vec:
    #normal = "0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001"
    sub['trend_items'] = sort_item2vec('trend_items', max_n=12)

In [ ]:
# sub['trend_items'] = remove_item2vec('trend_items', max_n=12)

In [ ]:
predicted = sub['trend_items'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Trend Items', ignore=(val_week == 105))
# Trend Items	0.013180	0.013154	0.013163	100.0

### 人気商品 ：属性、~年齢層（大まかな）~、オンライン／オフライン（４日前まで）グループは1,2,na<-1

In [ ]:
cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
cdf = cdf[['customer_id','age','attribute']]
#listBin = [-1, 19, 29, 39, 49, 59, 69, 119]
listBin = [-1, 19, 40, 119]
#cdf['cust_feature'] = pd.cut(cdf['age'], listBin).astype(str).str.cat(cdf['attribute'].astype(str))
cdf['cust_feature'] = cdf['attribute'].astype(str)
cdf = cdf.drop('age', axis=1)
cdf = cdf.drop('attribute', axis=1)
cdf['cust_feature'].unique()

In [ ]:
# # 4日間の売れた商品リスト
# init_date = last_date - dt.timedelta(days=4)
# sold_set = set(df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].article_id.tolist())
# len(sold_set)

In [ ]:
init_date = last_date - dt.timedelta(days=5 - 1)
group_df = pd.concat([cid, group.group.fillna(1)], axis=1) # grouping can be changed
group_df = group_df.merge(cdf, on='customer_id', how='right')
group_df.columns = ['customer_id', 'group', 'cust_feature']
group_df['cust_feature'] = group_df['group'].astype(str).str.cat(group_df['cust_feature'])
train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()\
    .merge(group_df, on='customer_id', how='left')\
    .groupby(['cust_feature', 'article_id']).t_dat.count().reset_index()

items = defaultdict(str)
for g in train.cust_feature.unique():
#    items[g] = iter_to_str(train.loc[train.cust_feature == g].sort_values('t_dat', ascending=False).article_id.tolist()[:30])
    items[g] = iter_to_str(train.loc[train.cust_feature == g].sort_values('t_dat', ascending=False).article_id.tolist()[:20])

sub['trend_items2'] = group_df.cust_feature.map(items)

In [ ]:
if use_item2vec:
    sub['trend_items2'] = sort_item2vec('trend_items2', max_n=12)

In [ ]:
predicted = sub['trend_items2'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Trend Items2', ignore=(val_week == 105))

### 人気商品 ： Active（４日前まで）グループは1,2,na<-1

In [ ]:
# cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
# cdf = cdf[['customer_id','attribute','Active']]
# cdf['cust_feature'] = cdf['attribute'].astype(str).str.cat(cdf['Active'].astype(str))
# cdf = cdf.drop('attribute', axis=1)
# cdf['cust_feature'].unique()

In [ ]:
# # 4日間の売れた商品リスト
# init_date = last_date - dt.timedelta(days=4)
# sold_set = set(df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].article_id.tolist())
# len(sold_set)

In [ ]:
# init_date = last_date - dt.timedelta(days=5 - 1)
# group_df = pd.concat([cid, group.group.fillna(1)], axis=1) # grouping can be changed
# group_df = group_df.merge(cdf, on='customer_id', how='right')
# group_df.columns = ['customer_id', 'group', 'Active', 'cust_feature']
# group_df['cust_feature'] = group_df['group'].astype(str).str.cat(group_df['cust_feature'])

# train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()\
#     .merge(group_df, on='customer_id', how='left')\
#     .groupby(['cust_feature', 'article_id']).t_dat.count().reset_index()

# items = defaultdict(str)
# for g in train.cust_feature.unique():
# #    items[g] = iter_to_str(train.loc[train.cust_feature == g].sort_values('t_dat', ascending=False).article_id.tolist()[:30])
#     items[g] = iter_to_str(train.loc[train.cust_feature == g].sort_values('t_dat', ascending=False).article_id.tolist()[:20])

# sub['trend_active'] = group_df.cust_feature.map(items)
# sub['trend_active'] = sub['trend_active'].fillna("")

In [ ]:
# cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
# cdf = cdf[cdf['Active'] == 1]
# cdf = cdf[['customer_id','attribute']]
# cdf.columns = ['customer_id', 'cust_feature']
# cdf['cust_feature'].unique()

cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
cdf = cdf[['customer_id','age','attribute','Active']]
#listBin = [-1, 19, 29, 39, 49, 59, 69, 119]
listBin = [-1, 19, 34, 49, 119]
cdf['cust_feature'] = pd.cut(cdf['age'], listBin).astype(str).str.cat(cdf['attribute'].astype(str))#.str.cat(cdf['Active'].astype(str))
#cdf['cust_feature'] = pd.qcut(cdf['age'], 10).astype(str).str.cat(cdf['attribute'].astype(str))
#cdf['cust_feature'] = pd.cut(cdf['age'], listBin).astype(str)
#cdf['cust_feature'] = pd.qcut(cdf['age'], 10).astype(str)
cdf = cdf.drop('age', axis=1)
cdf = cdf.drop('attribute', axis=1)
cdf = cdf.drop('Active', axis=1)

cdf['cust_feature'].unique()

In [ ]:
# # 4日間の売れた商品リスト
# init_date = last_date - dt.timedelta(days=4)
# sold_set = set(df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].article_id.tolist())
# len(sold_set)

In [ ]:
init_date = last_date - dt.timedelta(days=5 - 1)
group_df = pd.concat([cid, group.group.fillna(1)], axis=1) # grouping can be changed
group_df = group_df.merge(cdf, on='customer_id', how='left')
group_df.columns = ['customer_id', 'group', 'cust_feature']
group_df['cust_feature'] = group_df['group'].astype(str).str.cat(group_df['cust_feature'])

train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()\
    .merge(group_df, on='customer_id', how='left')\
    .groupby(['cust_feature', 'article_id']).t_dat.count().reset_index()

items = defaultdict(str)
for g in train.cust_feature.unique():
    items[g] = iter_to_str(train.loc[train.cust_feature == g].sort_values('t_dat', ascending=False).article_id.tolist()[:20])

sub['trend_ageatt'] = group_df.cust_feature.map(items)
sub['trend_ageatt'] = sub['trend_ageatt'].fillna("")

In [ ]:
if use_item2vec:
    sub['trend_ageatt'] = sort_item2vec('trend_ageatt', max_n=12)

In [ ]:
predicted = sub['trend_ageatt'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Trend age+attr', ignore=(val_week == 105))

### Woman 

In [ ]:
# cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
# #cdf = cdf[cdf['fashion_news_frequency'] >= 1]
# cdf = cdf[cdf['attribute'] == 'Woman']
# # cdf = cdf[cdf['Active'] == 1]
# cdf

In [ ]:
cdf = pd.read_parquet('../input/handmitem2vec/customers.parquet')
#cdf = cdf[cdf['fashion_news_frequency'] >= 1]
cdf = cdf[cdf['attribute'] == 'Woman']
# cdf = cdf[cdf['Active'] == 1]
cdf = cdf[['customer_id','age']]
# cdf.columns = ['customer_id', 'cust_feature']
# cdf['cust_feature'].unique()
#listBin = [-1, 19, 29, 39, 49, 59, 69, 119]
listBin = [-1, 19, 34, 49, 119]
#cdf['cust_feature'] = pd.cut(cdf['age'], listBin).astype(str).str.cat(cdf['Active'].astype(str))
#cdf['cust_feature'] = pd.qcut(cdf['age'], 10).astype(str).str.cat(cdf['attribute'].astype(str))
cdf['cust_feature'] = pd.cut(cdf['age'], listBin).astype(str)
#cdf['cust_feature'] = pd.qcut(cdf['age'], 10).astype(str)
cdf = cdf.drop('age', axis=1)
cdf['cust_feature'].unique()

In [ ]:
# # 4日間の売れた商品リスト
# init_date = last_date - dt.timedelta(days=4)
# sold_set = set(df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].article_id.tolist())
# len(sold_set)

In [ ]:
init_date = last_date - dt.timedelta(days=5 - 1)
group_df = pd.concat([cid, group.group.fillna(1)], axis=1) # grouping can be changed
group_df = group_df.merge(cdf, on='customer_id', how='left')
group_df.columns = ['customer_id', 'group', 'cust_feature']
group_df['cust_feature'] = group_df['group'].astype(str).str.cat(group_df['cust_feature'])

train = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].copy()\
    .merge(group_df, on='customer_id', how='left')\
    .groupby(['cust_feature', 'article_id']).t_dat.count().reset_index()

items = defaultdict(str)
for g in train.cust_feature.unique():
    items[g] = iter_to_str(train.loc[train.cust_feature == g].sort_values('t_dat', ascending=False).article_id.tolist()[:20])

sub['trend_woman'] = group_df.cust_feature.map(items)
sub['trend_woman'] = sub['trend_woman'].fillna("")

In [ ]:
if use_item2vec:
    sub['trend_woman'] = sort_item2vec('trend_woman', max_n=12)

In [ ]:
predicted = sub['trend_woman'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Trend Woman', ignore=(val_week == 105))

### item2vecだけでの予測

In [ ]:
# # 11日間の売れた商品リスト
# init_date = last_date - dt.timedelta(days=11)
# sold_set = set(df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].article_id.tolist())

In [ ]:
# l = []
# for i in tqdm(range(len(sub))) :
#     cid = sub.iloc[i]['customer_id']
#     if cid not in cust_buy :
#         l.append("")
#         continue
#     item = cust_buy[cid].split(" ")
#     r = model.wv.most_similar(item, topn=20)
#     t = []
#     for x, p in r :
#         if int(x) in sold_set :
#             t.append(x)
#     l.append(" ".join(t[:12]))
# #     if i == 10 : break

# sub['item2vec'] = l

In [ ]:
# predicted = sub['item2vec'].apply(lambda s: [] if pd.isna(s) else s.split())
# score = validation(actual, predicted, grouping, score, index='item2vec', ignore=(val_week == 105))

In [ ]:
# cnt = 0
# for i in range(len(actual)) :
    
#     if len(actual[i]) != 0 :
#         cid = sub['customer_id'][i]
# #         print(i, cid)
# #         print(actual[i])
#         if cid not in cust_buy : continue
#         item = cust_buy[cid].split(" ")
# #         print(item[-1])
#         r = model.wv.most_similar(item, topn=12)
#         t = []
#         for x, p in r :
#             if x in actual[i] :
#                 print("HIT", i, x)
#                 cnt+=1
#             if int(x) in sold_set :
#                 t.append(x)
# #         print(r, t)
# #         print(t)
# #         cnt+=1
# #         print("-"*40)
        
#     if i == 100000: break
# cnt/100000

## kangol

In [ ]:
# articles_df = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv",dtype=str, encoding='utf8')

In [ ]:
# xHM = []
# for i, e in enumerate(articles_df['detail_desc'].fillna("")) :
#     if 'Kangol x H&M' in e:
#         print(articles_df.iloc[i]['article_id'], e)
#         xHM.append(articles_df.iloc[i]['article_id'])
# #     elif 'Whooli Chen x H&M' in e :
# #         print(articles_df.iloc[i]['article_id'], e)
# #         xHM.append(articles_df.iloc[i]['article_id'])
        
# xHM


In [ ]:
# ll = " ".join(xHM[-12:])
# sub['kangol'] = [ll for _ in range(len(sub))]

In [ ]:
# sub['kangol'] = sort_item2vec('kangol', max_n=12)

In [ ]:
# predicted = sub['kangol'].apply(lambda s: [] if pd.isna(s) else s.split())
# score = validation(actual, predicted, grouping, score, index='kangol', ignore=(val_week == 105))

In [ ]:
#sub.head()

## 数年間定番アイテム

In [ ]:
# dayly sale

# 2018
day = 20
start = dt.datetime(2018,9,day,0,0)
end = dt.datetime(2018,9,day,23,59)
x = df.loc[(df.t_dat >= start) & (df.t_dat <= end)].article_id.tolist()

popular = set(x)
print(len(x))

for day in range(20, 31, 1) :
    start = dt.datetime(2018,9,day,0,0)
    end = dt.datetime(2018,9,day,23,59)
    x = df.loc[(df.t_dat >= start) & (df.t_dat <= end)].article_id.tolist()
    popular &= set(x)
    print(start, end, len(x), len(popular))

# 2019
for day in range(15, 30, 1) :
    start = dt.datetime(2019,9,day,0,0)
    end = dt.datetime(2019,9,day,23,59)
    x = df.loc[(df.t_dat >= start) & (df.t_dat <= end)].article_id.tolist()
    popular &= set(x)
    print(start, end, len(x), len(popular))

# 2020
for day in range(15, 23, 1) :
    start = dt.datetime(2020,9,day,0,0)
    end = dt.datetime(2020,9,day,23,59)
    x = df.loc[(df.t_dat >= start) & (df.t_dat <= end)].article_id.tolist()
    popular &= set(x)
    print(start, end, len(x), len(popular))



In [ ]:
from tqdm import tqdm
from collections import defaultdict

l = []
for i in tqdm(range(len(sub))):
    c_id = sub['customer_id'][i]
    if c_id not in cust_buy :
        l.append("")
        continue
        
    t = defaultdict(int)
    li = cust_buy[c_id].split(" ")
    #li.reverse()
#     print(cid, li)
    for e in li:
        if int(e) in popular :
            t[e] += 1
#     if len(t) != 0 : print(cid, t)
#     print(t)
    ti = []
    for e in t :
        ti.append([t[e],e])
    ti.sort(reverse=True)
    ti = list(map(lambda x: x[1], ti)) 
    l.append(" ".join(ti))
#     if i == 100: break
sub['popular_items'] = l 

In [ ]:
predicted = sub['popular_items'].apply(lambda s: [] if pd.isna(s) else s.split())
#score = validation(actual, predicted, grouping, index='anytime popular', ignore=(val_week == 105))
score = validation(actual, predicted, grouping, score, index='Popular Items', ignore=(val_week == 105))
#Popular Items	0.003576	0.002567	0.002901	3.4

## 嗜好の似たカスタマーが購入したもの

In [ ]:
if val_week == 105 :
    uucf = pd.read_csv('../input/hm-cf-data/uucf-2-0814.csv')
else :
    print("104")
#    uucf = pd.read_csv('../input/uucf104.csv')
    uucf = pd.read_csv('../input/hm-cf-data/uucf104-2-0814.csv')

uucf.head(10)

In [ ]:
l = []
for i in tqdm(range(len(uucf))) :
    s = uucf['prediction'][i]
    if s[0] == '[' :
        s = " ".join(eval(s))
    else : s = ""
    l.append(s)
sub['uucf'] = l
del uucf

In [ ]:
predicted = sub['uucf'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='uucf Items', ignore=(val_week == 105))
# 8352 9737 8653
# 0.011568

## another uucf

In [ ]:
from collections import defaultdict

def remove_(in_df, num = 10000) :
    m = defaultdict(int)
    target, tmax = "", 0
    for i in tqdm(range(len(in_df))) :
        p = in_df['prediction'][i]
        m[p] += 1
        if m[p] > tmax :
            tmax = m[p]
            target = p

    l = []
    skip = 0
    for i in tqdm(range(len(in_df))) :
        p = in_df['prediction'][i]
        if m[p] >= num:
            l.append("")
            skip += 1
        else :
            l.append(p)
    print("skip = ", skip)
    return l

In [ ]:
if val_week == 105 :
    uucf = pd.read_csv('../input/hm-cf-data/cf-v2.csv').fillna("")
else :
    print("104")
#    uucf = pd.read_csv('../input/uucf104.csv')
    uucf = pd.read_csv('../input/hm-cf-data/cf104-v2.csv').fillna("")

# uucf['prediction'] = remove_(uucf, 10)

sub['uucf2'] = uucf['prediction']
display(uucf.head(10))
del uucf

In [ ]:
predicted = sub['uucf2'].apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='uucf2 Items', ignore=(val_week == 105))
# 8352 9737 8653
# 0.011568 0.011552 0.011568

In [ ]:
sub

In [ ]:
# # 11日間の売れた商品リスト
# init_date = last_date - dt.timedelta(days=11)
# last_sell = df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)]
# sell =  last_sell.groupby('customer_id')\
#      .article_id.apply(iter_to_str).reset_index()\
#      .merge(cid, on='customer_id', how='right').article_id.fillna('')

In [ ]:
# last_buy = defaultdict(list)
# for i in tqdm(range(len(sub))) :
#     x = set(sell[i].split(" ")) - set('')
#     last_buy[sub['customer_id'][i]] = x

In [ ]:
# import pickle
# with open("../input/handmitem2vec/cust_similar.pkl", "rb") as f :
#     cust_similar = pickle.load(f)
# len(cust_similar)

In [ ]:
# l = []
# for i in tqdm(range(len(sub))): 
#     c = sub['customer_id'][i]
#     x = cust_similar[c]
#     y = set()
#     if c in cust_buy :
#         y = set(cust_buy[c].split(" "))
#     t = []
#     for e, _ in x :
#         if e in y : continue # 既に購入しているものはスキップ
#         if last_buy[e] == {''}: continue
#         t += last_buy[e]
#     l.append(" ".join(set(t)))

In [ ]:
# sub['similar_customers'] = l

In [ ]:
#if use_item2vec:
#    sub['similar_customers'] = sort_item2vec('similar_customers', max_n=12)

In [ ]:
# predicted = sub['similar_customers'].apply(lambda s: [] if pd.isna(s) else s.split())
# score = validation(actual, predicted, grouping, score, index='Similar Customers', ignore=(val_week == 105))

## save

In [ ]:
if is_save :
    print("SAVE.....")
    if val_week == 105: 
        sub.to_parquet('submission_all.parquet', index=False)
    else :
        sub.to_parquet('submission_all_104.parquet', index=False)

## memory clear

In [ ]:
# import gc
# del model
# del cust_buy
# gc.collect()

### 予測をブレンド

In [ ]:
# 11日間の売れた商品リスト
init_date = last_date - dt.timedelta(days=2)
sold_set = set(df.loc[(df.t_dat >= init_date) & (df.t_dat <= last_date)].article_id.tolist())

In [ ]:
# 11日間に販売実績のある商品だけ対象
# targets = ['last_purchase', 'popular_items', 'other_colors', 
#            'trend_woman', 'trend_ageatt', 'trend_items2', 'trend_items' ]
# weights = [1000, 5, 10, 
#            2, 2, 1.5, 1]


# targets = ['last_purchase', 'other_colors', 'popular_items', 
#            'trend_woman', 'trend_ageatt', 'trend_items2',  'uucf', 'trend_items' ]
# weights = [1000, 10, 5, 
#            1, 1, 1.3, 3, 0.5]

# targets = ['last_purchase', 'uucf', 'other_colors', 'popular_items', 
#            'trend_woman', 'trend_ageatt', 'trend_items2', 'trend_items' ]
# weights = [1000, 10, 10, 10, 
#            1, 1, 1.3, 0.5]
targets = ['last_purchase', 'other_colors', 'popular_items', 'uucf', 'uucf2', 'buy_together',
           'trend_woman', 'trend_ageatt', 'trend_items2', 'trend_items' ]
weights = [10, 2 ,2, 2, 2, 1, 
           1, 1, 1.3, 0.5]


sub['prediction'] = sub[targets].apply(blend, w=weights, axis=1, k=100).apply(prune, ok_set=sold_set)
predicted = sub.prediction.apply(lambda s: [] if pd.isna(s) else s.split())
score = validation(actual, predicted, grouping, score, index='Prediction', ignore=(val_week == 105))

#score が定義されていないとき（loadした時）
#score = validation(actual, predicted, grouping, index='Prediction', ignore=(val_week == 105))


#Prediction	0.019646	0.031350	0.027481	100.0 [100, 5, 10, 2, 1] without itemvec
#Prediction	0.020333	0.032050	0.028177	100.0 [100, 5, 10, 2, 1] with itemvec  

#Prediction	0.019675	0.031394	0.027520	100.0 [100, 5, 10, 2, 1, 2]
#Prediction	0.020852	0.032008	0.028320	100.0 [100, 5, 10, 2, 1, 2] test

#Prediction	0.019834	0.031329	0.027529	100.0
# 7 predictions with item2vec 
# first 3-items : 22754 23247 
#       4-items : 15229 15732 15836 15873
#     all items : 28536 
# 0.028359 0.028392 28464 0.028675 0.029264 0.029302 0.029368 0.029332 0.029333

In [ ]:
# targets = ['last_purchase', 'buy_together', 'other_colors', 'popular_items', 'uucf', 'uucf2']
# weights = [10, 1, 2, 2 ,2, 1]

# sub['customer'] = sub[targets].apply(blend, w=weights, axis=1, k=100).apply(prune, ok_set=sold_set)
# predicted = sub.customer.apply(lambda s: [] if pd.isna(s) else s.split())
# score = validation(actual, predicted, grouping, score, index='Customer', ignore=(val_week == 105))
# # 24625 0.024752 0.024641 0.024755 

In [ ]:
# targets = ['trend_woman', 'trend_ageatt', 'trend_items2', 'trend_items']
# weights = [1,1,1,1]
# sub['trend'] = sub[targets].apply(blend, w=weights, axis=1, k=100).apply(prune, ok_set=sold_set)
# predicted = sub.trend.apply(lambda s: [] if pd.isna(s) else s.split())
# score = validation(actual, predicted, grouping, score, index='Trend', ignore=(val_week == 105))

In [ ]:
# targets = ['customer', 'trend']
# weights = [4,1]
# sub['prediction'] = sub[targets].apply(blend, w=weights, axis=1, k=100).apply(prune, ok_set=sold_set)
# predicted = sub.prediction.apply(lambda s: [] if pd.isna(s) else s.split())
# score = validation(actual, predicted, grouping, score, index='Prediction', ignore=(val_week == 105))

In [ ]:
len(sub['prediction'][0].split())
#sns.barplot(data=score, x='All', y=score.index)

In [ ]:
if val_week == 105: sub[['customer_id', 'prediction']].to_csv('submission.csv', index=False)

## 全部埋まっているかチェック

In [ ]:
for i in tqdm(range(len(sub))) :
    x = sub['prediction'][i].split(" ")
    if len(x) != 12 :
        print(sub['customer_id'][i], x)
        break